In [1]:
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, plot, iplot

In [2]:
init_notebook_mode(connected=True)

In [3]:
iplot([{"x": [1, 2, 3], "y": [3, 1, 6]}])

In [4]:
%matplotlib inline

In [366]:
def get_range(mean,range_axes):
    lower = mean - range_axes
    upper = mean + range_axes
    lower = 1 if lower<=0 else lower
    return np.arange(lower,upper+1)

In [367]:
def get_prob_pdf(values, mean, std=3):
    prob = ss.norm.pdf(values,loc=mean,scale=std)
    prob = prob/prob.sum()
    return prob

In [368]:
def get_nums(array,probs,n_workflows=100):
    return np.random.choice(array,size=n_workflows,p=probs)

In [369]:
def get_normal_nums(x,range_axes):
    range_arr = get_range(x,range_axes)
    probs = get_prob_pdf(range_arr,x)
    length_arr = len(range_arr)
    return get_nums(range_arr,probs), length_arr

In [370]:
def get_pairs(x_mean,y_mean,range_axes = 7):
    return [get_normal_nums(i,range_axes) for i in [x_mean,y_mean]]

In [617]:
np.random.seed(150)

In [618]:
(x,x_len),(y,y_len) = get_pairs(6,4)

In [619]:
y = y*10

In [620]:
#y=y/x

In [680]:
workflow_ids = [f'Workflow-{i}' for i in range(100)]
drop_rate = (1-(y-np.min(y))/(np.max(y)-np.min(y)))
drops = drop_rate*y

In [731]:
def scatter_obj(x,y):
    return go.Scatter(
    x=x,
    y=y,
    text = workflow_ids,
        customdata=drops,
    hovertemplate=
        "<b>%{text}</b><br><br>" +
        "Initiations: %{y}<br>" +
        "Users Usage: %{x}<br>" +
        "Drops: %{customdata}"+
        "<extra></extra>",
    mode='markers',
    marker = dict(
        color = drop_rate,
        colorscale='bluered',
        colorbar=dict(thickness=20,title='DropRate'),
        size=14,
        opacity=0.4,
        #reversescale=True
    ),
    showlegend=False
    )

In [732]:
def vbar():
    return go.Bar(
    x=[6],
    y=[11],
    width = [0.1],
    name = '4 per User')

In [733]:
def hbar():
    return go.Bar(
    x=[13],
    y=[4],
    width = [0.2],
    orientation='h',
    name='6 per Usage')

In [734]:
def central_axis(lower,upper,orientation='h', color = '#1f77b4'):
    if orientation == 'h':
        xref = 'paper'
        yref = 'y'
        x0 = 0
        y0 = lower
        x1 = 1
        y1 = upper
    else:
        xref = 'x'
        yref = 'paper'
        x0 = lower
        y0 = 0
        x1 = upper
        y1 = 1

    return dict(
        type="rect",
        xref = xref,
        yref=yref,
        x0=x0,
        y0=y0,
        x1=x1,
        y1=y1,
        fillcolor = color,
        opacity=0.4,
        layer="above",
        line_width=0
    )

In [735]:
def rectangle_areas(x0, y0, x1, y1, color):
    return dict(
    type="rect",
        xref = "x",
        yref = "y",
        x0 = x0,
        y0 = y0,
        x1 = x1,
        y1 = y1,
        fillcolor = color,
        opacity=0.2,
        layer="below",
        line_width=0
    )

In [736]:
def rectangle_area_data(ranges_limits,axes_limits,quadrant):
    x_min,x_max,y_min,y_max = ranges_limits
    x_al,x_au,y_al,y_au = axes_limits
    return {
        'one':rectangle_areas(x_au,y_au,x_max,y_max,'#2ca02c'),
        'two':rectangle_areas(x_min,y_au,x_al,y_max,'darkorange'),
        'three':rectangle_areas(x_min,y_al,x_al,y_min,'indianred'),
        'four':rectangle_areas(x_au,y_min,x_max,y_al,'darkorange')
    }[quadrant]

In [737]:
def axes_limits(x,y):
    x_median = np.median(x)
    y_median = np.median(y)
    return x_median - 0.1, x_median + 0.1, y_median - 2, y_median + 2

In [738]:
def ranges_limits(x,y):
    return np.min(x), np.max(x), np.min(y), np.max(y)

In [739]:
def color_gradient(n_colors, start, end, opacity):
    colors = [Hcommon.get_format(i.get_rgb(), opacity) for i in
              start.range_to(get_color(end), n_colors)]
    return colors

In [740]:
# fig = go.Figure(
# data = [
#     scatter_obj(x,y),
#     vbar(),
#     hbar()
# ])

In [741]:
axes_limits_array = axes_limits(x,y)
x_al,x_au,y_al,y_au = axes_limits_array
ranges_limits_array = ranges_limits(x,y)

In [748]:
fig = go.Figure(
data = [
    scatter_obj(x,y)
])
fig.add_trace(go.Scatter(
x=[9,4],
y=[100,15],
text = ["High Usage Flows","Low Usage Flows"],
mode="text",
textfont=dict(
        color="black",
        size=15,
        family="Balto, sans-serif",
    ),
    showlegend=False
))
fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
},
    shapes = [
    central_axis(x_al,x_au,'v'),
    central_axis(y_al,y_au,'h'),
    rectangle_area_data(ranges_limits_array,axes_limits_array,'one'),
    rectangle_area_data(ranges_limits_array,axes_limits_array,'two'),
    rectangle_area_data(ranges_limits_array,axes_limits_array,'three'),
    rectangle_area_data(ranges_limits_array,axes_limits_array,'four')
],
xaxis_showgrid=False,
yaxis_showgrid=False,
xaxis_title = "No: of Users",
yaxis_title = "No: of Initiations",
title = "Quadrant Plot")
fig.show()